In [ ]:
import copy
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dbispipeline.analytics import get_results_as_dataframe
from mediaeval2021.analytics import extract_best_epoch
from mediaeval2021.analytics import extract_best_outcome
from mediaeval2021.analytics import extract_final_outcome
from mediaeval2021.analytics import extract_metrics
from mediaeval2021.analytics import plot_per_label
from mediaeval2021.analytics import print_per_label


results = get_results_as_dataframe(project_name='mediaeval2021', table_name='results')
results = results[results['date'] > '2021-10-13 12:00:00']
display(results['outcome'])

In [ ]:
# metric = 'f1_macro'

# results['final_outcome'] = results['outcome'].apply(extract_final_outcome)
# results['best_outcome'] = results['outcome'].apply(lambda v: extract_best_outcome(v, metric=metric))
# results['best_epoch'] = results['outcome'].apply(lambda v: extract_best_epoch(v, metric=metric))

def display_outcome(col):
    outcome = []
    for _, row in results.iterrows():
        out = copy.deepcopy(row[col])
        del out['confusion_matrix']
        del out['average_precision_all']
        del out['roc_auc_all']
        out['id'] = row['id']
        outcome.append(out)

    outcome = pd.DataFrame(outcome).merge(results[['id', 'sourcefile']], on=['id'])
    outcome['name'] = outcome['id'].apply(str) + ' ' + outcome['sourcefile']
    outcome[[
        'name',
#         'roc_auc',
        'f1_micro',
        'f1_macro',
        'average_precision',
        'precision_micro',
        'precision_macro',
    ]].plot.bar(x='name', title=col, figsize=(24,6))
    plt.show()

results = results.sort_values(by=['sourcefile'])
# display(results[['id', 'sourcefile', 'best_epoch']])

display_outcome('outcome')
best_outcome = pd.json_normalize(results['outcome'])
best_outcome['name'] = pd.DataFrame(results['id'].apply(str) + ' ' + results['sourcefile']).reset_index()[0]

paper_results = best_outcome[['name', 'roc_auc', 'average_precision', 'f1_micro', 'f1_macro']].round(3)
paper_results['name'] = paper_results['name'].apply(lambda v: v[4:].replace('plans/fixed_', '').replace('sampled_', '').replace('.py', '').replace('ensemble_c', 'ec'))
print(paper_results.to_latex(index=False))

In [ ]:
metric = 'pr-auc'

data593 = data = extract_metrics(results[results['id'] == 593].iloc[0]).sort_values(by=[metric])
print_per_label(data)
plt.show()

data594 = data = extract_metrics(results[results['id'] == 594].iloc[0]).sort_values(by=[metric])
print_per_label(data)
plt.show()

data602 = data = extract_metrics(results[results['id'] == 605].iloc[0]).sort_values(by=[metric])
print_per_label(data)
plt.show()

display(data.mean())

la = data.sort_values(by=[metric]).index[:28], data.sort_values(by=[metric]).index[28:]
print(la)

In [ ]:
data602['roc-auc593'] = data593['roc-auc']
data602['roc-auc594'] = data594['roc-auc']

print_per_label(data602[['roc-auc', 'roc-auc594', 'roc-auc593']].sort_values(by=['roc-auc594']))

In [ ]:
from mediaeval2021.analytics import plot_confusion_matrix

plot_confusion_matrix(results[results['id'] == 605].iloc[0], multilabel='heavy')